In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import re

In [9]:
def scrape_and_save_data():
    # Use the raw content URL
    url = "https://raw.githubusercontent.com/MarcusGrum/AIBAS/main/README.md"
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    }
    
    response = requests.get(url, headers=headers, timeout=10)
    content = response.text
    
    # Parse markdown table
    lines = content.split('\n')
    table_data = []
    in_table = False
    
    for line in lines:
        if '|' in line:
            # Extract cells from markdown table
            cells = [cell.strip() for cell in line.split('|')[1:-1]]
            
            # Skip separator rows (rows with only dashes and colons)
            if not all(re.match(r'^[-:\s]*$', cell) for cell in cells):
                table_data.append(cells)
                in_table = True
        elif in_table and '|' not in line:
            in_table = False
    
    if not table_data or len(table_data) < 2:
        print("No table found in the README.")
        return None
    
    # Convert the data into a DataFrame
    try:
        df = pd.DataFrame(table_data[1:], columns=table_data[0])
        df.to_csv("scraped_data.csv", index=False)
        print("Data scraped and saved successfully to scraped_data.csv")
        print(f"Shape: {df.shape}")
        print(df.head())
        return df
    except Exception as e:
        print(f"Error creating DataFrame: {e}")
        return None


df_raw = scrape_and_save_data()

Data scraped and saved successfully to scraped_data.csv
Shape: (10007, 2)
                    x                    y
0  10.611376985560522  -0.2581975455104324
1   4.386447954473255  0.00745422124225692
2   40.93916393492107   48.647258168949605
3   47.51486916723017    47.49779755650942
4  2.4025396488257247   1.4144366738030358


In [10]:
data = pd.read_csv('scraped_data.csv')
data.head()

,x,y
0,10.611376985560522,-0.258198
1,4.386447954473255,0.007454
2,40.93916393492107,48.647258
3,47.51486916723017,47.497798
4,2.4025396488257247,1.414437


In [11]:
# Remove empty rows and columns
data = data.dropna(how='all').dropna(axis=1, how='all')

# Convert numeric columns to appropriate types
for col in data.columns:
    try:
        data[col] = pd.to_numeric(data[col], errors='coerce')
    except ValueError:
        pass

# Handle missing values by filling numeric columns with their median
data = data.fillna(data.median(numeric_only=True))

# Display cleaned data
data.head()

,x,y
0,10.611377,-0.258198
1,4.386448,0.007454
2,40.939164,48.647258
3,47.514869,47.497798
4,2.402540,1.414437


In [12]:
# Calculate Z-scores for numeric columns to detect outliers
numeric_cols = data.select_dtypes(include=[np.number])
z_scores = (numeric_cols - numeric_cols.mean()) / numeric_cols.std()

# Remove rows where any numeric column has a Z-score > 3
data = data[(np.abs(z_scores) < 3).all(axis=1)]

# Display data after outlier removal
data.head()

/usr/local/lib/python3.12/dist-packages/pandas/core/nanops.py:1016: RuntimeWarning: overflow encountered in square
  sqr = _ensure_numeric((avg - values) ** 2)


,x,y
0,10.611377,-0.258198
1,4.386448,0.007454
2,40.939164,48.647258
3,47.514869,47.497798
4,2.402540,1.414437


In [13]:
# Normalize numeric columns using Min-Max scaling
for col in numeric_cols.columns:
    data[col] = (data[col] - data[col].min()) / (data[col].max() - data[col].min())

# Display normalized data
data.head()

,x,y
0,0.211976,6.694553e-159
1,0.087438,6.960205e-159
2,0.818725,5.560001e-158
3,0.950282,5.445055e-158
4,0.047747,8.367187e-159


In [14]:
# Save the cleaned and processed data to a CSV file
output_file = "UE_06_dataset04_joint_scraped_data.csv"
data.to_csv(output_file, index=False)

print(f"Data saved successfully to {output_file}")

Data saved successfully to UE_06_dataset04_joint_scraped_data.csv
